In [ ]:
import numpy as np
import os
from pandas import DataFrame
import pandas as pd
from sklearn.metrics import ndcg_score
from scipy import stats
import xlwt

def rnod(y_true, y_pred):
    indices = np.argsort(y_true)
    dist = np.abs(np.subtract(y_true[indices], y_pred[indices]))
    rnod_value = np.mean(dist) / np.sqrt(2)
    return rnod_value

def rnorm_sum_squares(x, y):
    x = np.array(x)
    y = np.array(y)
    assert x.shape == y.shape
    numerator = np.sum(np.square(x - y))
    denominator = np.sum(np.square(x) + np.square(y))
    rnorm_ss = np.sqrt(numerator / denominator)
    return rnorm_ss

def normalize_spearman(spearman):
    normalized = (spearman + 1) / 2
    return normalized
def normalize(p, q):
    p, q = np.asarray(p), np.asarray(q)
    assert (p >= 0).all(), p
    assert (q >= 0).all()
    p, q = p / p.sum(), q / q.sum()
    return p, q
def JSD(p, q, base=2):
    p, q = normalize(p, q)
    m = 1. / 2 * (p + q)
    return stats.entropy(p, m, base=base) / 2. + stats.entropy(q, m, base=base) / 2.
def result_metric(y_true, y_pred):
    metric = []
    ndcg = ndcg_score([y_true], [y_pred])
    Rjsd = 1 - JSD(y_true, y_pred, base=2)
    RRNSS = 1-rnorm_sum_squares(y_true, y_pred)
    metric.append(ndcg)
    metric.append(Rjsd)
    metric.append(RRNSS)
    return metric

def read_txt(trainset,testset,method):

    path1 = 'G:\\ML_result\DL+glove+feature'
    path6 = str(trainset) + '_'+ str(testset) + '_'+ str(method) + '.txt'
    path = os.path.join(path1, path6)
    print(path)
    file = open(path)

    val_list = file.readlines()
    truth_data_lists = []
    predicted_data_lists = []
    for string in val_list:
        string = string.split()
        truth_data_lists.append(float(string[0]))  
        predicted_data_lists.append(float(string[1]))
    truth_data = np.asarray(truth_data_lists) 
    predicted_data = np.asarray(predicted_data_lists)
    return truth_data, predicted_data


In [ ]:
methods = ['TEXT_ATT_BI_LSTM','TEXT_BI_LSTM']
datasets = ["CEFR", "CLEC", "CLOTH", "NES", "OSP", "RACE"]
trainsets= ["CEFR", "CLEC", "CLOTH", "NES", "OSP", "RACE"]

for method in methods:
    workexcel = xlwt.Workbook(encoding='ascii') 
    writer = pd.ExcelWriter(
        'G:\\ML_result\DL+glove+feature\cor\\cor'+ str(method) + '.xlsx',
        engine='xlsxwriter')

    for trainset in trainsets: 
        testset_metric = []
        for testset in datasets: 
            truth_data, predicted_data = read_txt(trainset,testset,method) 
            data = result_metric(truth_data, predicted_data)
            print(data)
            testset_metric.append(data)
            df = DataFrame(columns=['NDCG','RJSD','RRNSS'], data=testset_metric)
            df.to_excel(writer, sheet_name=trainset, index=False)
    writer.save()